In [1]:
import pandas as pd
import csv

In [2]:
import _pickle as cPickle
import import_ipynb
import pandas as pd
import numpy as np
from genUidStat import loadData,genUidStat
from evaluation import precision
from runTime import runTime
from pathos.pools import _ProcessPool
from multiprocess.pool import Pool

importing Jupyter notebook from evaluation.ipynb
importing Jupyter notebook from runTime.ipynb


In [3]:
df1=pd.read_csv("weibo_train1.csv")
df2=pd.read_csv("weibo_train2.csv")
frames=[df1,df2]
traindata=pd.concat(frames)

In [4]:
def splitDataFrameIntoSmaller(df, chunkSize = 10000): 
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf

In [5]:
uid_stat=pd.read_csv("train_uid_stat.csv")

In [6]:
uid = splitDataFrameIntoSmaller(uid_stat, chunkSize = 500)

In [7]:
uid[0].shape[0]

500

In [8]:
def search_all_uid(stat_dic,file):
	import pandas as pd
	import numpy as np
	def _deviation(predict, real, kind):
		t = 5.0 if kind=='f' else 3.0
		return abs(predict - real) / (real + t)
	def _precision_i(fp, fr, cp, cr, lp, lr):
		return 1 - 0.5 * _deviation(fp, fr, 'f') - 0.25 * _deviation(cp, cr, 'c') - 0.25 * _deviation(lp, lr, 'l')
	def _sgn(x):
		return 1 if x>0 else 0
	def _count_i(fr, cr, lr):
		x = fr + cr + lr
		return 101 if x>100 else (x+1)
	def precision(real_and_predict):
		numerator,denominator = 0.0,0.0
		for  fr, cr, lr,fp, cp, lp in real_and_predict:
			numerator += _count_i(fr, cr, lr) * _sgn(_precision_i(fp, fr, cp, cr, lp, lr) - 0.8)
			denominator += _count_i(fr, cr, lr)
		return (numerator / denominator)
	def score(uid_data,pred):
		"""
		uid_data:
			pd.DataFrame
		pred:
			list, [fp,cp,lp]
		"""
		uid_real_pred = uid_data[['forward_count','comment_count','like_count']]
		uid_real_pred['fp'] = pred[0]
		uid_real_pred['cp'] = pred[1]
		uid_real_pred['lp'] = pred[2]
		return precision(uid_real_pred.values)
	def search(uid_data,target,args):
		args = list(args)
		target_index = ['forward_count','comment_count','like_count'].index(target)
		target_min,target_median,target_max = args[3*target_index:3*target_index+3]
		del args[3*target_index:3*target_index+3]
		pred = (args[1],args[4])
		
		best_num = [target_median]
		best_pred = list(pred)
		best_pred.insert(target_index,target_median)
		best_score = score(uid_data,best_pred)
		for num in range(target_min,target_max+1):
			this_pred = list(pred)
			this_pred.insert(target_index,num)
			this_score = score(uid_data,this_pred)
			if this_score >= best_score:                  
				if this_score > best_score:
					best_num = [num]
					best_score = this_score
				else:
					best_num.append(num)                       
				
		return best_num[np.array([abs(i - target_median) for i in best_num]).argmin()]
	uid_best_pred = {}
	pool = _ProcessPool()
	uids,f,c,l = [],[],[],[]
	m=1
	for uid in stat_dic:
		print ("search uid:{}".format(uid),m)
		m=m+1
		uid_data = traindata[traindata.u_id == uid]
		arguments = stat_dic[uid][['forward_min','forward_median','forward_max','comment_min',\
					'comment_median','comment_max','like_min','like_median','like_max']]
		arguments = tuple([int(i) for i in arguments]) 
		f.append(pool.apply_async(search,args=(uid_data,'forward_count',arguments)))
		c.append(pool.apply_async(search,args=(uid_data,'comment_count',arguments)))
		l.append(pool.apply_async(search,args=(uid_data,'like_count',arguments)))
		uids.append(uid)
	pool.close()
	pool.join()
	f = [i.get() for i in f]
	c = [i.get() for i in c]
	l = [i.get() for i in l]
	for i in range(len(uids)):
		uid_best_pred[uids[i]] = [f[i],c[i],l[i]]
	#cPickle.dump(uid_best_pred,open('uid_best_pred'+str(file)+'.pkl','ab'))
	label = ['forward_count','comment_count','like_count']
	pd.DataFrame.from_dict(data=uid_best_pred,orient='index').to_csv("G:\\Anconda Prog\\BestPred\\weibo_uidbest"+str(file)+".csv",header=label)
	print("Written to file")



In [ ]:
uid_stat=pd.read_csv("train_uid_stat.csv")
uid_stat=uid_stat.set_index('u_id')
uid = splitDataFrameIntoSmaller(uid_stat, chunkSize = 100)
n=8
while n<75:
    df=uid[n].T
    stat=df.to_dict('series')
    n=n+1
    search_all_uid(stat,n)

search uid:059e69f515a8ccae9005d9184082e1a7 1
search uid:059f1af2f4d0cc0c11f7c2333fb56df9 2
search uid:05a17e58a7d0318c02ca957ea287c63a 3
search uid:05a3e00d1bf81123eaa16d9140781814 4
search uid:05a86ee2ef6ca329c2d7bd29a0bd43a2 5
search uid:05aa7401f543aff4eaa9804faf94fe5d 6
search uid:05abd155225287be6ccc9e851743c33d 7
search uid:05ac93f1cea114840ec678882df58bde 8
search uid:05ac976dd6c437b557aacc0f8bf95820 9
search uid:05ad91adac4f36090687b821074a6839 10
search uid:05b00857a652495ecd61ff287eefa0fa 11
search uid:05b0ed6b6c5a3c7ec0ee133658afc455 12
search uid:05ba03da99f9a8fd86ce5cedabb74eaa 13
search uid:05ba689e3f3d89f4f1de80b156f09c51 14
search uid:05bab1fcb0bef33fd6ab88f718d5d41d 15
search uid:05bc20ee20f50b744c00d948da2ee82f 16
search uid:05bc29c0517afb7cb63379e447646db2 17
search uid:05bc52673524ae6b2342f8c00e815aa8 18
search uid:05bd6785b6c3ca20728be79ed7e2fd73 19
search uid:05be804ef16a4d1b3b442cc1668c15ca 20
search uid:05bf76486b0c1eac3fab100d88678514 21
search uid:05c1a7c2eec

# FILES GENERATED

In [92]:
@runTime
def predict_by_search(submission=True):
	traindata,testdata = loadData()
	#concat all frames here
	
	
	ub=pd.read_csv("train_best_pred.csv")
	ub=ub.set_index('u_id')
	df=ub.T
	uid_best_pred=df.to_dict('series')
	#uid_best_pred = search_all_uid()
	#print ("search done,now predict on traindata and testdata...")

	#predict traindata with uid's best fp,cp,lp
	forward,comment,like = [],[],[]
	for uid in traindata['u_id']:
		if uid in uid_best_pred:
			forward.append(int(uid_best_pred[uid][0]))
			comment.append(int(uid_best_pred[uid][1]))
			like.append(int(uid_best_pred[uid][2]))
		else:
			forward.append(0)
			comment.append(0)
			like.append(0)
	#score on the traindata
	train_real_pred = traindata[['forward_count','comment_count','like_count']]
	train_real_pred['fp'],train_real_pred['cp'],train_real_pred['lp'] = forward,comment,like
	print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))
	if submission:
		test_pred = testdata[['u_id','m_id']]
		forward,comment,like = [],[],[]
		for uid in testdata['u_id']:
			if uid in uid_best_pred:
				forward.append(int(uid_best_pred[uid][0]))
				comment.append(int(uid_best_pred[uid][1]))
				like.append(int(uid_best_pred[uid][2]))
			else:
				forward.append(0)
				comment.append(0)
				like.append(0)
		test_pred['fp'],test_pred['cp'],test_pred['lp'] = forward,comment,like
		
		#generate submission file
		result = []
		filename = "weibo_predict_search.txt"
		for _,row in test_pred.iterrows():
			result.append("{0}\t{1}\t{2},{3},{4}\n".format(row[0],row[1],row[2],row[3],row[4]))
		f = open(filename,'w')
		f.writelines(result)
		f.close()
		print ('generate submission file "{}"'.format(filename))

In [93]:
predict_by_search()

search uid:24b621c98f2594b698c0b1d60c9ae6db
search uid:d38e9bed5d98110dc2489d0d1cac3c2a
search uid:d80f3d3c5c1d658e82b837a4dd1af849
search uid:da534fe87e7a52777bee5c30573ed5fd
search uid:e06a22b7e065e559a1f0bf7841a85c51
search uid:e44d81d630e4f382f657e72aa4b685da
search uid:f349a67d1cd7c8683c5bbc5f8486e193
search uid:f9828598f9664d4e347ef2048ce17734
search uid:fa13974743d3fe6ff40d21b872325e9e
search uid:fbe6c953632e1b3dda66cf6118b6ab12
1
Before loop

search done,now predict on traindata and testdata...


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Score on the training set:55.56%


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


generate submission file "weibo_predict_search.txt"
predict_by_search run time: 72.50s
